In [16]:
import numpy as np
import skimage
import utils
import pathlib

In [17]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a threshold value

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    distinct_intensity_levels = 2**8  # L

    height, width = im.shape
    number_of_pixels = height * width

    # Compute the normalized histogram
    normalized_histogram = np.zeros(distinct_intensity_levels)
    for y in range(height):
        for x in range(width):
            normalized_histogram[im[y, x]] += 1 / number_of_pixels

    # Compute the cumulative sums
    cumulative_sums = np.zeros(distinct_intensity_levels)
    cumulative_sums[0] = normalized_histogram[0]
    for i in range(1, distinct_intensity_levels):
        cumulative_sums[i] = cumulative_sums[i - 1] + normalized_histogram[i]

    # Compute the cumulative means
    # Compute the cumulative means
    cumulative_means = np.zeros(distinct_intensity_levels)
    for i in range(1, distinct_intensity_levels):
        cumulative_means[i] = cumulative_means[i - 1] + (i * normalized_histogram[i])

    # Compute the global mean
    global_mean = cumulative_means[distinct_intensity_levels - 1]

    # Compute the between-class variance term
    between_class_variance_term = np.zeros(distinct_intensity_levels)
    for i in range(distinct_intensity_levels):
        num = (global_mean * cumulative_sums[i] - cumulative_means[i]) ** 2
        denom = cumulative_sums[i] * (1 - cumulative_sums[i])

        if denom == 0:
            between_class_variance_term[i] = 0
        else:
            between_class_variance_term[i] = num / denom

    # Obtain the Otsu threshold, k*
    max_variance = max(between_class_variance_term)
    indices = np.where(between_class_variance_term == max_variance)[0]
    threshold = round(sum(indices) / len(indices))

    return threshold

In [18]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("polymercell.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == np.bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

Reading image: images/thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed/thumbprint-segmented.png


/tmp/ipykernel_793/1997791817.py:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert segmented_image.dtype == np.bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(


Reading image: images/polymercell.png
Found optimal threshold: 181
Saving image to: image_processed/polymercell-segmented.png
